In [ ]:
import numpy as np
import pandas as pd
import multiprocessing

from scipy import integrate
import matplotlib.pyplot as plt
import scipy.stats as stats
import math
from scipy import optimize

from sklearn.linear_model import LinearRegression


import matplotlib.pyplot as plt
import bokeh
import bokeh.io
from bokeh.plotting import figure
from bokeh.io import output_notebook, show

# init_notebook_mode()

import seaborn as sns

import re
import math
import copy

from collections import defaultdict
import csv
import itertools
import datetime 
from datetime import datetime
import time
import dateutil.parser
import pickle
import random

import gc
import zipfile
import sys, getopt
import os

from IPython.core.interactiveshell import InteractiveShell
from io import StringIO

import dask.dataframe as dd
#from chest import Chest

InteractiveShell.ast_node_interactivity = "all"
#InteractiveShell.ast_node_interactivity = "last"

# Magic function to make matplotlib inline
%matplotlib inline

%config InlineBackend.figure_formats = {'png', 'retina'}

# Set up Bokeh for inline viewing
bokeh.io.output_notebook()

import dask.dataframe as ddf
import dask.array as da

pd.set_option('max_columns', 500)
pd.set_option('max_rows', 800)

In [ ]:
# Normal density plot
plt.rcParams["figure.figsize"] = [10, 5]

mu = 38000
# variance = 1
sigma = 7000 # math.sqrt(variance)
x = np.linspace(mu - 3.5*sigma, mu + 3.5*sigma, 100)
plt.plot(x, stats.norm.pdf(x, mu, sigma), linewidth = 4)

plt.title('CONTIENTAL 80 Prediction Distribution', size = 18)
plt.xlabel('Net Demand Quantity', size = 18)
plt.ylabel('Probability Density', size = 18)

### e.g. Stan Smith: Normal prediction distribution

In [ ]:
# e.g. Stan Smith Prediction Distribution 

plt.rcParams["figure.figsize"] = [6, 3]

mu = 38000
sigma = 5000 # math.sqrt(variance)
x = np.linspace(mu - 4*sigma, mu + 4*sigma, 100)
plt.plot(x, stats.norm.pdf(x, mu, sigma), linewidth = 4)

plt.title('Stan Smith Prediction Distribution', size = 12)
plt.xlabel('Net Demand Quantity', size = 12)
plt.ylabel('Probability Density', size = 12)

## Optimization functions

In [ ]:
# Loss --- demand, buy, margin, cost
def L(d, b, margin, cost):
    if d > b:
        return (d - b)*margin
    elif d < b:
        return (b - d)*cost
    elif d == b:
        return 0
    else:
        print('Error')

# E[L | buy, article_mean, article_sd, article_margin, article_cost]
def EL(b_0, mu_0, sigma_0, margin_0, cost_0):
    I = lambda x: L(x, b_0, margin_0, cost_0) * stats.norm.pdf(x, mu_0, sigma_0) # I for integrand
    Exp_loss = integrate.quad(I, mu_0 - 4*sigma_0, mu_0 + 4*sigma_0)
    return round(Exp_loss[0], 2) 


# return buy_qty that minimizes expected loss
def min_expected_loss(mu, sigma, margin, cost):
    
    buys = list(range(mu - 1*sigma, mu + 3*sigma, 50)) # buy qtys for which calculate E[L]
    ELs = [EL(b, mu, sigma, margin, cost) for b in buys] # E[L|b] for b in buys  
    min_loss_index = ELs.index(min(ELs)) # index of buy qty that minimizes E[L]
    
    return buys[min_loss_index]          # buy qty that minimized E[l]
    

In [ ]:
# Stan Smith: Prediction, E[L] minimizing buy, comparison 

mu = 38000
sigma = 7000
margin = 65.80
cost = 4.20

buys = list(range(mu - 1*sigma, mu + 3*sigma, 50)) # buy qtys for which calculate E[L]
ELs = [EL(b, mu, sigma, margin, cost) for b in buys]

print('Buying', buys[ELs.index(min(ELs))], 'units minimizes expected loss') # buy qty that minimizes expected loss
print()
print('Minimized expected loss:', min(ELs)) # minimized Expected loss
print()
print('Expected loss with 20% buffer approach:', EL(38000*1.2, 38000, 7000, 65.80, 4.2))
print()
print('Expected profit increase:', round(EL(buys[ELs.index(min(ELs))], 38000, 7000, 65.80, 4.20) - EL(38000*1.2, 38000, 7000, 65.80, 4.2), 2))

# Expected loss against function of buy qty
plt.rcParams["figure.figsize"] = [12,8]
plt.plot(buys, ELs, linewidth = 3)

plt.title('Expected Loss vs. Buy Quantity', size = 18)
plt.xlabel('Buy Quantity', size = 18)
plt.ylabel('Expected Loss', size = 18)

min_expected_loss(mu, sigma, margin, cost)

### Comparing Distributions

In [ ]:
X = np.random.normal(loc=0.0, scale=1.0, size=1000) # NORMAL
plt.hist(X, density = True)

x = np.linspace(-4, 4, 100)
plt.plot(x, stats.norm.pdf(x, 0, 1), linewidth = 4)

In [ ]:
# Kolmogorov-Smirnov test
stats.kstest(X, 'norm')

In [ ]:
from numpy.random import randn
from statsmodels.graphics.gofplots import qqplot
from matplotlib import pyplot

qqplot(X, line='s')

### Asymmetric prediction distribution; e.g. Continental 80s

In [ ]:
# c = np.random.gamma(shape = 2, scale = 200, size = 1000) + 600 # GAMMA
# plt.hist(c, density = True, bins = 50)

shape = 5
scale = 2000
shift = 38000 - shape*scale

x = np.linspace(0, shape*scale + 5*math.sqrt(shape*scale**2), 100) + shift
plt.plot(x, stats.gamma.pdf(x - shift, a = shape, scale = scale), linewidth = 4)

plt.title('Skewed Distribution (shifted Gamma(shape = 5, scale = 2000))', size = 18)
plt.xlabel('Net Demand Quantity', size = 18)
plt.ylabel('Density', size = 18)

In [ ]:
# ------ GAMMA EXPECTED LOSS -------
# We think demand will be Gamma(shape, scale), with mean shape*scale, variance shape*scale^2
# For article with cost = cost_0, margin = margin_0
 
import math

# Expected value of Loss function, given: buy, shape, scale, margin, cost, non-centrality
def EL_gamma(b_0, shape_0, scale_0, margin_0, cost_0, shift_0):
    
    I = lambda x: L(x, b_0, margin_0, cost_0) * stats.gamma.pdf(x - shift_0, a = shape_0, scale = scale_0) # I for integrand
    
    Exp_loss = integrate.quad(I, 0 + shift_0, shift_0 + shape_0*scale_0 + 5*math.sqrt(shape_0*scale_0**2))
    
    return round(Exp_loss[0], 2) 


In [ ]:
# Plot hypothetical Expected loss, function of buy qty
shape = 5
scale = 2000
shift = 38000 - shape*scale

margin = 65.80
cost = 4.20

buys = list(range(0 + shift, shift + shape*scale + 4*np.int(math.sqrt(shape*scale**2)), 50)) # buy qtys to for which it calculates E[L]
ELs = [EL_gamma(b, shape, scale, margin, cost, shift) for b in buys]


print('buying', buys[ELs.index(min(ELs))], 'units minimizes expected loss')
print()
print('The minimized expected loss is:', min(ELs)) # minimized Expected loss
print()
print('Expected loss with 20% buffer approach:', EL_gamma(38000*1.2, shape, scale, margin, cost, shift))
print()
print('Expected profit increase:', EL_gamma(buys[ELs.index(min(ELs))], shape, scale, margin, cost, shift) - EL_gamma(38000*1.2, shape, scale, margin, cost, shift))

# print('Loss of', L(38000, buys[ELs.index(min(ELs))], 65.80, 4.20), 'if we buy optimally and predict perfectly') # loss assoc. with that buy if prediction perfect

plt.rcParams["figure.figsize"] = [12,8]
plt.plot(buys, ELs, linewidth = 3)

plt.title('Expected Loss vs. Buy Quantity', size = 18)
plt.xlabel('Buy Quantity', size = 18)
plt.ylabel('Expected Loss', size = 18)



## Big Fish fishing

In [ ]:
# B28128

mu = 800
sigma = 200
margin = 55 - 11
cost = 11

buys = list(range(mu - 1*sigma, mu + 3*sigma, 50)) # buy qtys for which calculate E[L]
ELs = [EL(b, mu, sigma, margin, cost) for b in buys]

print('Buying', buys[ELs.index(min(ELs))], 'units minimizes expected loss') # buy qty that minimizes expected loss
print()
print('Minimized expected loss:', min(ELs)) # minimized Expected loss
print()
print('Expected loss with 20% buffer approach:', EL(mu*1.2, mu, sigma, margin, cost))
print()
print('Expected profit increase:', 
      round(np.abs(EL(buys[ELs.index(min(ELs))], mu, sigma, margin, cost) - EL(mu*1.2, mu, sigma, margin, cost)), 2))

plt.rcParams["figure.figsize"] = [12,8]
plt.plot(buys, ELs, linewidth = 3)

plt.title('Expected Loss vs. Buy Quantity', size = 18)
plt.xlabel('Buy Quantity', size = 18)
plt.ylabel('Expected Loss', size = 18)

### Framework Extension EDA: combine information

In [ ]:
x = np.random.normal(loc=1000, scale=100, size=1000) # NORMAL
c = np.random.gamma(shape = 2, scale = 200, size = 1000) + 600 # GAMMA

plt.hist((x, c), bins = 20, density = True)
plt.title('Combining Information', size = 18)
plt.xlabel('Hypothetical Buy Quantity Data Points', size = 18)

In [ ]:
xc = np.concatenate((x, c)) # NORMAL + GAMMA
pd.DataFrame(xc).quantile((0.05, 0.95))
plt.hist(xc, bins = 40, density = True, color = 'skyblue')
plt.axvline(700, linestyle='dashed', linewidth=3)
plt.axvline(1324, linestyle='dashed', linewidth=3)
plt.axvline(xc.mean(), color = 'orange', linestyle='dashed', linewidth=3)

plt.title('Combining Information: Empirical Mean and Confidence Intervals', size = 18)
plt.xlabel('Hypothetical Buy Quantity Data Points', size = 18)

# EDA: SD Estimation Impact 

In [ ]:
# minimize Expected loss over buy quantity b_0

# Change: b_0 must come last for use with partial() function
def EL_b0(mu_0, sigma_0, margin_0, cost_0, b_0):
    I = lambda x: L(x, b_0, margin_0, cost_0) * stats.norm.pdf(x, mu_0, sigma_0) # I for integrand
    Exp_loss = integrate.quad(I, mu_0 - 4*sigma_0, mu_0 + 4*sigma_0)
    return round(Exp_loss[0], 2) 

def minimize_EL(mu, sigma, margin, cost):
    
    p = partial(EL_b0, mu, sigma, margin, cost) # Make EL function of only one var: b_0
    buy_opt = optimize.minimize_scalar(p, bounds = (mu - sigma, mu + 3*sigma))
    
    return int(buy_opt['x']), int(buy_opt['fun'])

In [186]:
# Remember: to calculate the EL under an incorrect sigma0 estimate:
    # (1) Calculate the optimal buy under true sigma, and assoc loss
    # (2) Calculate the optimal buy under falst sigma0
    # (3) Calculate the expected losses under true mu, sigma --- but with the b_0 from optimizing w/ sigma0 in (2)

### Consider again Continental 80s
* margin = 65.80
* cost = 4.20
* Suppose true d ~ N(38000, 7000)
* Suppose incorrectly think d ~ N(38000, ?)

In [207]:
mu = 38000
sigma = 7000
margin = 65.80
cost = 4.20

a = minimize_EL(mu, sigma, margin, cost)

# Believed optimal and true EL
print('sigma_0:', 5000)
b = minimize_EL(mu, 5000, margin, cost)
b, EL(mu, 7000, margin, cost, b[0]) - EL(mu, 7000, margin, cost, a[0])

# True optimal and EL:
print('sigma:', sigma)
a

# Believed optimal and believed EL
print('sigma_0:', 9000)
b = minimize_EL(mu, 9000, margin, cost)
b, round(EL(mu, 7000, margin, cost, b[0]) - EL(mu, 7000, margin, cost, a[0])) 

# Takeaway: more expensive to underestimate SD than overestimate SD

sigma_0: 5000


((45772, 41661), 7211.5)

sigma: 7000


(48880, 58325)

sigma_0: 9000


((51990, 74990), 4570)

In [190]:
mu = 38000
sigma = 7000
margin = 65.80
cost = 4.20

a = minimize_EL(mu, sigma, margin, cost) # minimize with real sigma
sigma, a

for sig0 in range(1000, 10001, 500):
    b = minimize_EL(mu, sig0, margin, cost)  # buy qty to minimize with fake sigma
    EL_0 = EL(mu, sigma, margin, cost, b[0]) # EL with that buy quantity, but real sigma
    c = round(EL_0 - a[1])
    print('sig_0 =', sig0, '-- increase in expected losses:', c)

(7000, (48880, 58325))

sig_0 = 1000 -- increase in expected losses: 94028
sig_0 = 1500 -- increase in expected losses: 76040
sig_0 = 2000 -- increase in expected losses: 60310
sig_0 = 2500 -- increase in expected losses: 46782
sig_0 = 3000 -- increase in expected losses: 35330
sig_0 = 3500 -- increase in expected losses: 25780
sig_0 = 4000 -- increase in expected losses: 18032
sig_0 = 4500 -- increase in expected losses: 11903
sig_0 = 5000 -- increase in expected losses: 7212
sig_0 = 5500 -- increase in expected losses: 3847
sig_0 = 6000 -- increase in expected losses: 1617
sig_0 = 6500 -- increase in expected losses: 385
sig_0 = 7000 -- increase in expected losses: 1
sig_0 = 7500 -- increase in expected losses: 340
sig_0 = 8000 -- increase in expected losses: 1282
sig_0 = 8500 -- increase in expected losses: 2722
sig_0 = 9000 -- increase in expected losses: 4571
sig_0 = 9500 -- increase in expected losses: 6743
sig_0 = 10000 -- increase in expected losses: 9186


In [194]:
a = minimize_EL(mu, sigma, margin, cost)
print('sigma =', sigma, '-- (optimal buy, EL):', a)
print()

sigma = 7000 -- (optimal buy, EL): (48880, 58325)



(48880, 58325)

In [198]:
print('sigma0, EL increase')

results = [(sig0, a[1] - EL(minimize_EL(mu, sig0, margin, cost)[0]) for sig0 in range(1000, 15001, 1000)]

sigma0, EL increase


In [ ]:
# Plot: 
# x-axis: sigma_0
# y-axis: losses over optimal

## Estimate prediction SD

In [ ]:
dat = pd.read_csv('weekly_sales_all.csv', low_memory=False, index_col = 0) # *** DATA ***


In [ ]:
dat.head()
dat.shape

In [ ]:
dat2 = pd.DataFrame(dat.groupby(['article_number', 'season'])['net_qty'].sum())

In [ ]:
dat3 = dat2.groupby('article_number').aggregate(['mean', 'std', 'max', 'count'])

In [ ]:
dat3.head()
dat3.dropna(inplace=True)

dat3 = dat3[dat3['net_qty']['max'] < 50000]

dat3.head()

In [ ]:
X = np.array(dat3['net_qty']['max']).reshape(-1, 1)
y = np.array(dat3['net_qty']['std'])

In [ ]:
reg = LinearRegression().fit(X, y)

In [ ]:
reg.score(X, y)
reg.coef_

reg.intercept_ 
reg.predict([[38000]])

reg.coef_*38000 + reg.intercept_

### Plot SD against mean

In [ ]:
dat_max = dat3['net_qty']['max']
dat_std = dat3['net_qty']['std']

dat_max.head()

In [ ]:
plt.rcParams["figure.figsize"] = [12,9]
plt.scatter(dat3['net_qty']['max'], dat3['net_qty']['std'])

plt.title('Standard Deviation vs. Max Season Net Demand Qty', size = 18)
plt.xlabel('Max Season Net Demand Qty', size = 18)
plt.ylabel('Standard Deviation', size = 18)

### Optimal buy for SDs

In [ ]:
sigs = np.linspace(100, 700, 100)
opt_buys = [min_expected_loss(1000, np.int(s), 85, 15) for s in sigs]

In [ ]:
plt.plot(sigs, opt_buys, linewidth = 3)

plt.title('Optimal Buy vs. Prediction Standard Dev.', size = 18)
plt.xlabel('Standard Deviation', size = 18)
plt.ylabel('Optimal Buy', size = 18)

plt.rcParams["figure.figsize"] = [18, 9]


## New Article EDA

In [ ]:
dat = pd.read_csv('weekly_sales_all.csv', low_memory=False, index_col = 0) # *** DATA ***


In [ ]:
dat2 = pd.DataFrame(dat.groupby(['article_number', 'year', 'season'])['net_sales'].sum())

In [ ]:
dat2.reset_index(inplace=True)

In [ ]:
dat_ref = pd.read_csv('dat_ref.csv')
dat_ref = dat_ref[['group_article', 'rmh_product_division_descr', 'rmh_category_descr', 'rmh_retail_section_descr', 'rmh_product_type_descr']]

In [ ]:
dat3 = pd.merge(dat2, dat_ref, left_on='article_number', right_on='group_article')

In [ ]:
len(set(dat2.article_number).intersection(set(dat_ref.group_article)))

dat2.shape
dat3.shape

len(dat3.article_number.unique())

In [ ]:
# Retrieve some SS19 only articles, to then build empirical-integrated priors for
dat00 = dat2.reset_index()

dat000 = pd.crosstab(index=dat00["article_number"],  # Make a crosstab
                              columns=dat00['season']).sum(axis = 1)